# Unit Testing Tutorial

In this tutorial, we will see the Julia's built-in unit testing functionality.
This kind of functionality exists in one way or another for essentially all high-level programming languages.
But, we'll focus on Julia in this tutorial.

Our tutorial is taken largely from the Julia documentation, which I encourage you to read at this [link](https://docs.julialang.org/en/v1/stdlib/Test/).

## The `Test` Package

The `Test.jl` package contains functionality for unit testing.
So, in order to use it we need to load it.

In [ ]:
using Test

# this will hide stacktraces
# see https://discourse.julialang.org/t/suppress-stacktrace-when-a-test-fails/19834/7
Test.eval(quote
    function record(ts::DefaultTestSet, t::Union{Fail, Error})
        push!(ts.results, t)
    end
end)

## Basic Unit Tests

Simple unit testing (i.e. one-off tests) can performed with the `@test` macro.
Here's the basic use cases:

You will run `@test test_ex` and

- if `test_ex` evaluates to `true` then the result is `Test Passed`
- if `test_ex` evaluates to `false` then the result is `Fail Result`
- if `test_ex` cannot be evaluated, then the result is `Error Result`

Let's see this in action.

In [ ]:
@test true

In [ ]:
@test 3 + 4 == 7

In the cases above, the test passes and we get a nice green text telling us this.

Let's see what happens when the test fails

In [ ]:
@test false

A slightly more interesting example might be more enlightening.

In [ ]:
@test 3 + 4 == 9

So when the test fails, we see a few things:

- We see **Test Failed** in bold red letters -- can't miss this one
- We see the expression as it was stated, i.e. `3 + 4 == 9`
- We see which values were evaluated, i.e. `7 == 9`
- We see a Stacktrace, which tells us where the error is coming from. Admittedly, that's not helpful because we're using a Jupyter notebook. But once we "clean up" the tests, it will become handy.

In [ ]:
@test 3.0 + [3,4] # this won't be able to evaluate

### Example: Testing Numerical Equality

Almost all of statistical computing involves dealing with numerical values, often floating point numbers.
For example, your estimated parameter will usually be a floating point number of a array thereof.

You might want to tell if two floating point numbers are equal, e.g. the estimator your code spits out and what you expect the answer to be.
However, due to roudning error, two floating point numbers will usually never be equal, even if you expect them to be.

Let's see the simplest example.

In [ ]:
x = 0.2
y = 0.1
z = 0.3

# mathematically, this is true
x + y == z

In [ ]:
z - (x+y) # but, this value is very very small, i.e. numerically zero

Isn't that frustrating? But, that's the reality of working with floating point arithmetic -- we can't possibly store every real number on the computer (after all, there are a lot more real numbers than bits on our machines) and so we have to accept tiny rounding errors every once and a while.

Point is: when testing numerical equality, you'll need an *inexact equality test*.
This is the purpose of the built-in `isapprox` function in Julia.
I will tell you a bit about this function, but here are more resources:

- Documentation [here](https://docs.julialang.org/en/v1/base/math/#Base.isapprox)
- Discourse discussion [here](https://discourse.julialang.org/t/approximate-equality/8952).
- Great example of a clean docstring in the [source code](https://github.com/JuliaLang/julia/blob/01a2eadb0474c395845e66ed3382b52e0c1f1b8f/base/floatfuncs.jl#L158).

The function `isapprox` takes the following inputs

```julia
isapprox(x, y; atol::Real=0, rtol::Real=atol>0 ? 0 : √eps, nans::Bool=false[, norm::Function])

```
- The function `isapprox` returns `true` if `norm(x-y) <= max(atol, rtol*max(norm(x), norm(y)))`.
- The default `atol` (absolute tolerance) is zero and the default `rtol` (relative tolerance) depends on the types of `x` and `y`.

Let's give this a whirl.

In [ ]:
x = 0.2
y = 0.1
z = 0.3

isapprox(x+y, z)

But remember, this all depends on the default values of `rtol` and `atol`:

In [ ]:
isapprox(x+y, z , rtol=1e-20)

The default tolerance parameters are usually okay, but you always want to give this careful consideration.
For example, do you want these two values to be the same?

In [ ]:
x = [10.0^8, 1.0]
y = [10.0^8, 2.0]

isapprox(x,y)

Sometimes, using `isapprox` is quick and simple, but it does pay off to carefully consider any numerical tests and the scales that you are expecting.

### Back to Unit Testing

Let's see how you can use this in a unit test.
The key feature that I want to highlight here is that you can optional arguments to the `@test` function.

More precisely, you can write `@test f(args...) key=val...`

In [ ]:
x = 0.2
y = 0.1
z = 0.3

@test isapprox(x+y, z)

In [ ]:
@test isapprox(x+y, z) rtol=1e-20

### Testing for Errors

Sometimes you want to check that your code does indeed throw certain errors when supplied incorrect inputs.

The general syntax is `@test_throws exception expr` where

- The test returns `Test Passed` if evaluating `expr` throws `exception`
- The rest returns `Test Failed` if evaluating `expr` does not throw `exception`

Let's see a few of these in action.

In [ ]:
@test_throws BoundsError [1,2,3][4]

In [ ]:
@test_throws DimensionMismatch [1,2,3] + [4,5]

In [ ]:
@test_throws DimensionMismatch [1,2] + [3,4]

You will probably use the these `@test_throws` less than `@test`, but it can still be helpful:

- You may have code that expects covariate matrix`X` is `n`-by-`d`. If you supply a matrix `X` that is `d`-by-`n`, you want to see that your code throws an error, e.g. `DimensionMismatch`.

## Working with Test Sets

In general, you will have multiple unit tests.
The problem with the `@test` macro is that it throws an exception immediately as soon as a test fails.
This is a problem because you typically want to know the result of all of the tests.

Here's an example:

In [ ]:
@test 4 + 5 == 9
@test length("so long and thanks for all the fish") == 6
@test 6 * 9 == 42

The problem is that we didn't get to find out whether our code for computing `42` worked.
The series of tests above just stopped as soon as it saw an error in the calculation of the length of `"so long and thanks for all the fish"`.

To overcome this issue, we can use the `@test_set` macro, which groups tests into sets.
All of the tests in a test set will be fun, and at the end of the test set a summary will be printed.
If any of the tests failed, or could not be evaluated due to an error, the test set will then throw a `TestSetException`.

The simplest syntax works as follows:

In [ ]:
@testset "Douglas Adams Tests" begin
    @test 4 + 5 == 9
    @test length("so long and thanks for all the fish") == 6
    @test 6 * 9 == 42
end;

So now we can run all the tests and see that two of the tests failed.
Let's see what is looks like when all the tests pass.

In [ ]:
@testset "trigonometric identities" begin
   θ = 2/3*π
   @test sin(-θ) ≈ -sin(θ) # this is shorthand for isapprox
   @test cos(-θ) ≈ cos(θ)
   @test sin(2θ) ≈ 2*sin(θ)*cos(θ)
   @test cos(2θ) ≈ cos(θ)^2 - sin(θ)^2
end;

### Nested Test Sets

In fact, test sets can be nested.
If we use the `verbose = true` option, then we can see the results of sub-tests.

Let's see what this looks like.

In [ ]:
foo(x) = length(x)^2

@testset verbose=true "Foo Tests" begin
    @testset "Animal Test" begin
        @test foo("cat") == 9
        @test foo("dog") == foo("cat")
    end
    
    @testset "Vegetable Test" begin
        @test foo("corn") == 16
        @test foo("apple") == 25
    end
end;

### Looping Tests

We can also use the `@testset` macro to create a sequence of tests via a `for` loop.

In [ ]:
@testset "Arrays $i" for i in 1:5
   @test foo(zeros(i)) == i^2
   @test foo(fill(1.0, i)) == i^2
end;

But, this is just a *sequence* of tests in the sense that if one of them fails, the `@testset` will stop.

In [ ]:
@testset "Arrays $i" for i in 1:5
   @test foo(zeros(i)) == i^2
   @test foo(fill(1.0, i == 4 ? 3 : i)) == i^2
end;

Note that we didn't get to the `i=5` test above.

If you want to see a summary for all of the tests in the loop, you need to wrap it again in a `@testset` macro.

In [ ]:
@testset verbose=true begin
    @testset "Arrays $i" for i in 1:5
        @test foo(zeros(i)) == i^2
        @test foo(fill(1.0, i == 4 ? 3 : i)) == i^2
#         @test foo(fill(1.0, i)) == i^2
    end
end;

As a final silly example, let's try to determine whether the student (our program) is correctly adding, or whether they are [quadding](https://en.wiktionary.org/wiki/quaddition).

In [ ]:
function quaddition(x,y)
    if max(x,y) >= 57
        return 5
    else
        return x+y
    end
end

@testset begin
    @testset "Quadding Check Tests: $x + y" for x in 50:60
        @test quaddition(x,1) == x + 1
        @test quaddition(x,5) == x+ 5
        @test quaddition(x,10) == x + 10
    end;
end

## Taking A Step Back

We have now seen how to use the `@test` and `@testset` macros to define unit tests.

But unfortunately Jupyter notebooks are a **horrible** way to run unit tests!
It's not quick or easy *at all*.

Instead, we should take a different approach.
You will want to keep three separate directories:

- `src`: this contains your source code
- `test`: this contains your test code
- `notebooks`: this contains your Jupyter notebooks

Let's continue the tutorial in this format!